In [14]:
import json
from utility.model_mf import MFModelCP, MFModelILP, MFSModelCP, MFSModelILP
import time
import plotly.express as px
import pandas as pd


In [15]:
with open("data/model.json") as json_file:
    data = json.load(json_file)



In [16]:
list_jobs = []
list_constr = []
for name_job in data['nodes']:
    res = data['nodes'][name_job]['properties']['Resource']['value']
    product_instance = data['nodes'][name_job]['properties']['Product_instance']['value']
    duration = data['nodes'][name_job]['properties'][f'{name_job}_duration']['value']
    list_jobs.append([name_job,res,product_instance,duration])

for name_constr in data['edges']:
    type_constr = data['edges'][name_constr]['__type__']
    list_constr.append([name_constr,type_constr])

In [19]:
df_jobs = pd.DataFrame(list_jobs, columns=['name', 'product', 'resource','duration'])
df_jobs

,name,product,resource,duration
0,1_16_0_0,Source_Source,1_16_0,10.0
1,1_16_0_1,Cell_0_Transport,1_16_0,10.0
2,1_16_0_2,Cell_0_Carrousel,1_16_0,8.0
3,1_16_0_3,Cell_0_Staubli,1_16_0,33.0
4,1_16_0_4,Cell_0_Binpick,1_16_0,320.0
...,...,...,...,...
100,warehouse_for_1_16_4_6,Cell_0_Warehouse_1,1_16_4,NaN
101,filler_between_warehouse_for_1_16_4_5_and_ware...,Cell_0_Warehouse_1,1_16_4,NaN
102,warehouse_for_1_16_4_8,Cell_0_Warehouse_1,1_16_4,NaN
103,filler_between_warehouse_for_1_16_4_6_and_ware...,Cell_0_Warehouse_1,1_16_4,NaN


In [20]:
df_edge = pd.DataFrame(list_constr, columns=['name', 'constraint type', ])
df_edge

,name,constraint type
0,1_16_0_0_is_right_before_1_16_0_1,IsRightBefore
1,1_16_0_1_is_right_before_1_16_0_2,IsRightBefore
2,filler_between_1_16_0_2_and_warehouse_for_1_16...,IsRightAfter
3,filler_between_1_16_0_2_and_warehouse_for_1_16...,IsRightBefore
4,warehouse_for_1_16_0_3_is_synced_1_16_0_3,IsSynced
...,...,...
944,filler_between_warehouse_for_1_16_4_5_and_ware...,HasNoOverlap
945,filler_between_warehouse_for_1_16_4_5_and_ware...,HasNoOverlap
946,warehouse_for_1_16_4_8_has_no_overlap_filler_b...,HasNoOverlap
947,warehouse_for_1_16_4_8_has_no_overlap_filler_b...,HasNoOverlap


In [21]:
model_ilp = MFModelILP(data)

for solver in ['ortools','exact','gurobi','choco']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 1.2080557346343994 seconds and 2764 of makespan
Solving for exact with ILP formulation: 10.928021669387817 seconds and 2764 of makespan
Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2025-04-18
Solving for gurobi with ILP formulation: 1.3639607429504395 seconds and 2764 of makespan
Timeout reached for choco with ILP formulation


In [22]:
model_ilp = MFModelILP(data)
start = time.time()
for solver in ['ortools']:
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 1.626615047454834 seconds and 2764 of makespan


In [23]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration = model_ilp.get_dv()
to_show =  []

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    resource = info[job_info['__id__']][1]
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)

fig.show()

In [11]:
with open("data/data_max_parallel_new.json") as json_file:
    data = json.load(json_file)

In [12]:
model_ilp = MFModelILP(data, parallel=True)
for solver in ['ortools']:
    start = time.time()
    if model_ilp.solve(solver=solver,time_limit=60):
        end = time.time()
        print(f'Solving for {solver} with ILP formulation: {end-start} seconds and {model_ilp.get_makespan()} of makespan')
    else:
        print(f'Timeout reached for {solver} with ILP formulation')

Solving for ortools with ILP formulation: 6.732336521148682 seconds and 828 of makespan


In [13]:
jobs = model_ilp.get_jobs()
info = model_ilp.get_info()
start, duration, job_resource,_ = model_ilp.get_dv()
info_resource = model_ilp.get_info_resource()
to_show =  []
dict_job_start = {}

for index in range(len(start)):
    dict_job_start[index] = start[index].value()

for job,job_info in jobs.items():
    subject = info[job_info['__id__']][0]
    start_job = start[subject].value()
    end_job = start[subject].value() + duration[subject].value()
    ris_id = job_resource[subject].value()
    resource = next((k for k, v in info_resource.items() if v == ris_id), None)
    product = info[job_info['__id__']][2]
    job_name = info[job_info['__id__']][4]
    to_show.append(dict(Task = f'Task {subject}', Start= start_job,End = end_job, Resource = resource, Product = product, Name=job_name))

# Convert the tasks to a DataFrame for easier manipulation
df = pd.DataFrame(to_show)

# Calculate the duration of each task
df['Duration'] = df['End'] - df['Start']

# Use px.bar to create the Gantt chart with item-based colors
fig = px.bar(df, x='Duration', y='Resource', color='Product', orientation='h',
             hover_data=['Start', 'End', 'Task','Name'], base='Start', title="Job Shop Schedule")

# Update the layout to set the background to black and adjust text/grid colors for visibility
fig.update_layout(
    plot_bgcolor='black',  # Set background color of the plot to black
    paper_bgcolor='black',  # Set the overall figure background to black
    xaxis=dict(showgrid=True, gridcolor='lightgray', title="Time", color='white'),  # Set grid lines and label color to light for visibility
    yaxis=dict(showgrid=True, gridcolor='lightgray', title="Machine", color='white', categoryorder='category ascending'),  # Set grid and label color
    font=dict(color='white'),  # Set general font color to white for readability
    title_font=dict(color='white')  # Ensure the title font is also visible
)
fig.update_layout(height=750)
fig.show()